In [1]:
#single file design approved by LUIS

'''from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/Team Drives/EE461P"'''

'from google.colab import drive\ndrive.mount(\'/content/drive\')\n!ls "/content/drive/Team Drives/EE461P"'

In [2]:
!pip install mlxtend
!pip install xgboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from xgboost import XGBRegressor, XGBClassifier
from matplotlib.colors import LinearSegmentedColormap
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

    100% |████████████████████████████████| 1.3MB 4.8MB/s eta 0:00:01
    100% |████████████████████████████████| 13.8MB 767kB/s eta 0:00:01
    100% |████████████████████████████████| 15.0MB 519kB/s eta 0:00:01
    100% |████████████████████████████████| 16.8MB 704kB/s eta 0:00:01
    100% |████████████████████████████████| 24.5MB 478kB/s eta 0:00:01
    100% |████████████████████████████████| 8.3MB 658kB/s eta 0:00:01
    100% |████████████████████████████████| 235kB 15.4MB/s ta 0:00:01
    100% |████████████████████████████████| 112kB 12.9MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 8.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/f8/0e/2365ddc010afb3d79147f1dd544e5ee24bf4ece58ab99b16fbb465ce6dc0/pytz-2018.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/67/4b/141a581104b1f6397bfa78ac9d43d8ad29a7ca43ea90a2d863fe3056e86a/six-1.11.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 645kB 3.9MB/s 

ModuleNotFoundError: No module named 'xgboost'

In [3]:
#student_data = pd.read_csv("/content/drive/Team Drives/EE461P/hsls_16_student_v1_0.csv")
student_data = pd.read_csv("hsls_16_student_v1_0.csv")

In [4]:
def remove_neg(column):
    new_column = []
    for x in range(len(column)):
        if column[x] < 0:
            continue
        new_column.append(column[x])
    return new_column

#for continuous variables
def impute(column):
    sorted_column = column.sort_values()
    median_arr = []
    for x in sorted_column.keys():
        if sorted_column[x] < 0:
            continue
        median_arr.append(sorted_column[x])
    
    median = 0
    split_index = int(len(median_arr)/2)
    mod = len(median_arr)%2
    if mod == 1:
        median = median_arr[split_index]
    else:
        median = (median_arr[split_index] + median_arr[split_index-1])/2
    
    for x in column.keys():
        if column[x] == -8 or column[x] == -9:
            column[x] = median
    return column

def one_hot_encode(column,categories):
    for x in column.keys():
        if column[x] < 0:
            column[x] = np.nan
    if len(categories) == 2:
        binary_arr = []
        for x in column.keys():
            if column[x] == categories[0]:
                column[x] = int(1)
            else:
                column[x] = int(0)
        return pd.DataFrame({'1.0':column})
    else:
        return pd.get_dummies(column)

def is_target(target,targets):
    for t in targets:
        if target == t:
            return True
    return False
    
def impute_encode(df,threshold,targets):
    for column in df:
        if(is_target(column,targets)):
            continue
        unique_arr = df[column].unique()
        unique_arr = remove_neg(unique_arr)
        if len(unique_arr) > threshold:
            imputed_column = impute(df[column])
            df[column] = imputed_column
        else:
            encoded_column = one_hot_encode(df[column],unique_arr)
            df.drop(column,axis=1,inplace=True)
            index = 0
            for x in encoded_column.columns:
                df[str(column)+'_'+str(int(index))] = encoded_column[x]
                index += 1
    return

## <p style="text-align: center;">Clean Data</p>

In [5]:
#remove suppressed data and weights
array = []
for x in student_data.columns:
    if student_data[x][0] == -5:
        array.append(x)
        
student_no_suppressed_data = student_data.drop(columns = array)

In [6]:
#remove weight data from their models
array = []
for x in student_no_suppressed_data.columns:
    if x[0] == "W":
        array.append(x)
        
student_no_suppressed_or_weights = student_no_suppressed_data.drop(columns = array)

In [7]:
#student_no_suppressed_or_weights.to_csv("/content/drive/Team Drives/EE461P/no_suppressed_and_no_weights.csv")

In [8]:
#do manual feature selection
X_and_y = pd.read_csv("x and y Columns.csv")
student_reduced_columns = student_no_suppressed_or_weights[X_and_y['X'].tolist() + X_and_y['y'].dropna().tolist()]
student_reduced_columns.head()

,X1SEX,X1RACE,X1HISPANIC,X1WHITE,X1BLACK,X1DUALLANG,X1TXMTSCOR,X1TXMPROF1,X1TXMPROF2,X1TXMPROF3,...,X3TGPASOCST,X3TGPAART,X3TGPALANG,X3TGPAACAD,X3TAFGPATOT,X4HSCOMPSTAT,X4EVRAPPCLG,X4PS1SELECT,X4PS1DEGTYPE1,X4PSLFSTFB16
0,1,8,0,1,0,1,59.3710,0.9997,0.9669,0.8976,...,4.0,4.0,3.5,3.5,3.5,1,1,5,2,4
1,2,8,0,1,0,1,47.6821,0.9714,0.5204,0.1768,...,4.0,-1.0,4.0,4.0,4.0,1,1,2,1,5
2,2,3,0,0,1,1,64.2431,1.0000,0.9914,0.9838,...,2.5,4.0,3.0,2.5,2.5,1,1,3,1,2
3,2,8,0,1,0,1,49.2690,0.9843,0.6416,0.2459,...,4.0,4.0,4.0,4.0,4.0,1,-9,-7,-7,5
4,1,8,0,1,0,1,62.5897,0.9999,0.9863,0.9686,...,3.5,3.5,3.0,3.5,3.5,1,1,-7,-7,8


In [ ]:
#hot encode and impute data
student_encoded_and_imputed = student_reduced_columns.copy(deep=True)

threshold = 10
targets = X_and_y['y'].dropna().tolist()
impute_encode(student_encoded_and_imputed,threshold,targets)

In [ ]:
#scale the data
student_encoded_and_imputed_scaled_data = student_encoded_and_imputed

## <p style="text-align: center;">Remove Highly Correlated Features and Targets </p>

In [ ]:
#coloring variables for the two heatmaps
min_color = 'white'
max_color = (0.03137254, 0.18823529411, 0.41960784313, 1)
cmap = LinearSegmentedColormap.from_list("", [min_color, max_color])

In [ ]:
#split data into y and x
X = student_encoded_and_imputed_scaled_data[X_and_y['X'].tolist()]
y = student_encoded_and_imputed_scaled_data[X_and_y['y'].dropna().tolist()]

In [ ]:
#find y correlation matrix
corr_sorted = abs(y.corr())
sorted_df = y[list(corr_sorted.index)]
y_corr = round(sorted_df.corr(), 2)

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))

fig = sns.heatmap(y_corr, annot=True, cmap=cmap,
                  xticklabels=y_corr.columns.values,
                  yticklabels=y_corr.columns.values,
                  ax = ax)
plt.xticks(rotation=90)
fig.xaxis.set_tick_params(labelsize=8)
fig.yaxis.set_tick_params(labelsize=8)

plt.show()

From the data above we see that the target features are neatly separated into two highly correlated sections. We are going to use the feature, X3TGPAACAD, in the top left box for our regression part of our ensemble. We are going to use X4PS1SELECT from the bottom right correlated box for our classification part.

In [ ]:
y_columns_to_keep = ['X3TGPAACAD', 'X4PS1SELECT']

In [ ]:
#find x correlation matrix
corr_sorted = abs(X.corr())
sorted_df = X[list(corr_sorted.index)]
X_corr = round(sorted_df.corr(), 2)

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))

fig = sns.heatmap(X_corr, annot=True, cmap=cmap,
                  xticklabels=X_corr.columns.values,
                  yticklabels=X_corr.columns.values,
                  ax = ax)
plt.xticks(rotation=90)
fig.xaxis.set_tick_params(labelsize=8)
fig.yaxis.set_tick_params(labelsize=8)

plt.show()

In [ ]:
#find the highly correlated features to eliminate
corr_matrix = pd.DataFrame(X_corr)
corr_col = corr_matrix.columns

counter = pd.DataFrame([[0]*len(corr_col)], columns = corr_col)
count_of_pairs = 0

for x in range(0, len(corr_col)):
    for i in range(x+1, len(corr_col)):
        if abs(X_corr[corr_col[x]][corr_col[i]]) > .90:
            count_of_pairs = count_of_pairs + 1
            counter[corr_col[x]][0] = counter[corr_col[x]][0] + 1
            counter[corr_col[i]][0] = counter[corr_col[i]][0] + 1
            print("(", corr_col[x],",", corr_col[i], ") with a correlation of ", X_corr[corr_col[x]][corr_col[i]])
            
print("The amount of highly correlated pairs is", count_of_pairs)

In [ ]:
#eliminate features that are too highly correlated
X_columns_to_keep = corr_col[(counter < 10).iloc[0, :]].values
X_columns_to_keep.shape

In [ ]:
#Refine data to not have overly correlated features

cleaned_data = student_encoded_and_imputed_scaled_data[np.append(X_columns_to_keep, y_columns_to_keep)]
cleaned_data.shape

In [ ]:
def drop_neg_rows(df, column):
    mask = df[column] < 0
    return df.drop(index=mask.index[mask])

In [ ]:
#split into Train and Test for the regression
reg_target = y_columns_to_keep[0]

reg_data = cleaned_data[np.append(X_columns_to_keep, reg_target)]
reg_data = drop_neg_rows(reg_data, reg_target)
print(reg_data.shape)

X_reg = reg_data[X_columns_to_keep]
y_reg = reg_data[reg_target]

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.33, random_state=1)

In [ ]:
#split into Train and Test for the classification
class_target = y_columns_to_keep[1]

class_data = cleaned_data[np.append(X_columns_to_keep, class_target)]
class_data = drop_neg_rows(class_data, class_target)
print(class_data.shape)

X_class = class_data[X_columns_to_keep]
y_class = class_data[class_target]

X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(
    X_class, y_class, test_size=0.33, random_state=1)

## <p style="text-align: center;">Using Regression to find the most important features</p>

In [ ]:
def LinearRegressionFeatureSelection(n_features):
    lin_reg = SFS(LinearRegression(),
          k_features=n_features,
          forward=True,
          floating=False,
          verbose=2,
          scoring='neg_mean_squared_error',
          cv=3)
    lin_reg.fit(X_reg_train, y_reg_train)
    time.sleep(1)
    
    lin_reg_selected_features = LinearRegression()
    lin_reg_selected_features.fit(X_reg_train[list(lin_reg.subsets_[n_features]['feature_names'])],y_reg_train)
    
    features = pd.DataFrame({'Features': lin_reg.subsets_[n_features]['feature_names'], 'Coefs': lin_reg_selected_features.coef_})
    display(features)
    
    lin_reg_fig = plot_sfs(lin_reg.get_metric_dict(), kind='std_err')
    plt.show()
    return features

In [ ]:
def LassoRegressionFeatureSelection(n_features):
    lasso = SFS(Lasso(alpha=.1),
          k_features=n_features,
          forward=True,
          floating=False,
          verbose=2,
          scoring='neg_mean_squared_error',
          cv=3)
    lasso.fit(X_reg_train, y_reg_train)
    time.sleep(1)
    
    lasso_reg_selected_features = Lasso(alpha = .1)
    lasso_reg_selected_features.fit(X_reg_train[list(lasso.subsets_[n_features]['feature_names'])],y_reg_train)
    
    features = pd.DataFrame({'Features': lasso.subsets_[n_features]['feature_names'], 'Coefs': lasso_reg_selected_features.coef_})
    display(features)
    
    lin_reg_fig = plot_sfs(lasso.get_metric_dict(), kind='std_err')
    plt.show()
    return features

In [ ]:
def XGBRegGBLinearFeatureSelection(n_features):
    xgb = XGBRegressor(booster = 'gblinear')
    xgb.fit(X_reg_train, y_reg_train)
    
    feature_importance = pd.DataFrame({'Features': X_reg_train.columns, 'Coefficients': np.array(xgb.coef_).astype(np.float), 'abs(coefs)': abs(np.array(xgb.coef_).astype(np.float))})
    feature_importance.sort_values(by=['abs(coefs)'], ascending = False, inplace = True)
    feature_importance.drop(feature_importance.tail(len(feature_importance.index)-n_features).index, inplace=True)
    
    display(feature_importance)
    return feature_importance

In [ ]:
def XGBRegGBTreeFeatureSelection(n_features):
    xgb = XGBRegressor(booster = 'gbtree')
    xgb.fit(X_reg_train, y_reg_train)
    
    feature_importance = pd.DataFrame({'Features': X_reg_train.columns, 'Coefficients': np.array(xgb.feature_importances_).astype(np.float)})
    feature_importance.sort_values(by=['Coefficients'], ascending = False, inplace = True)
    feature_importance.drop(feature_importance.tail(len(feature_importance.index)-n_features).index, inplace=True)
    
    display(feature_importance)
    return feature_importance

In [ ]:
#REGRESSION
linear_reg_features = LinearRegressionFeatureSelection(15)

In [ ]:
lasso_reg_features = LassoRegressionFeatureSelection(15)

In [ ]:
xgb_linear_reg_features = XGBRegGBLinearFeatureSelection(10)

In [ ]:
xgb_tree_reg_features = XGBRegGBTreeFeatureSelection(10)

## <p style="text-align: center;">Using Classification to find the most important features</p>

In [0]:
def XGBClassifierFeatureSelection(n_features):
    xgb = XGBClassifier()
    xgb.fit(X_class_train, y_class_train)
    
    feature_importance = pd.DataFrame({'Features': X_class_train.columns, 'Coefficients': np.array(xgb.feature_importances_).astype(np.float)})
    feature_importance.sort_values(by=['Coefficients'], ascending = False, inplace = True)
    feature_importance.drop(feature_importance.tail(len(feature_importance.index)-n_features).index, inplace=True)
    
    display(feature_importance)
    return feature_importance

In [0]:
def RandomForestClassifierFeatureSelection(n_features):
    rf = RandomForestClassifier()
    rf.fit(X_class_train, y_class_train)
    
    feature_importance = pd.DataFrame({'Features': X_class_train.columns, 'Coefficients': np.array(rf.feature_importances_).astype(np.float)})
    feature_importance.sort_values(by=['Coefficients'], ascending = False, inplace = True)
    feature_importance.drop(feature_importance.tail(len(feature_importance.index)-n_features).index, inplace=True)
    
    display(feature_importance)
    return feature_importance

In [0]:
#CLASSIFICATION

#xgbclassifier
#randomforest
#naive bayes

In [0]:
xgb_class_features = XGBClassifierFeatureSelection(10)

In [0]:
random_forest_class_features = RandomForestClassifierFeatureSelection(10)